In [1]:
import warnings
warnings.filterwarnings('ignore')

# Chapter 5. Matrices and equations

## The LU decomposition

### Implementing the LU decomposition

In [2]:
import numpy as np


def elimination_matrix(
    A: np.ndarray,
    step: int, 
):
    """
    Computes the step-th elimination matrix and its inverse.
    
    Args:
        A (np.ndarray): The matrix of shape (n, n) for which 
            the LU decomposition is being computed.
        step (int): The current step of elimination, an integer
            between 1 and n-1

    Returns:
        elim_mtx (np.ndarray): The step-th elimination matrix
            of shape (n, n)
        elim_mtx_inv (np.ndarray): The inverse of the
            elimination matrix of shape (n, n)
    """
    
    n = A.shape[0]
    elim_mtx = np.eye(n)
    elim_mtx_inv = np.eye(n)
    
    if 0 < step < n:
        a = A[:, step-1]/A[step-1, step-1]
        elim_mtx[step:, step-1] = -a[step:]
        elim_mtx_inv[step:, step-1] = a[step:]
    
    return elim_mtx, elim_mtx_inv

In [3]:
def LU(A: np.ndarray):
    """
    Computes the LU factorization of a square matrix A.
    
    Args:
        A (np.ndarray): A square matrix of shape (n, n) to be factorized. 
            It must be non-singular (invertible) for the 
            decomposition to work.

    Returns:
        L (np.ndarray): A lower triangular matrix of shape (n, n) 
            with ones on the diagonal.
        U (np.ndarray): An upper triangular matrix of shape (n, n).
    """
    
    n = A.shape[0]
    L = np.eye(n)
    U = np.copy(A)
    
    for step in range(1, n):
        elim_mtx, elim_mtx_inv = elimination_matrix(U, step=step)
        U = np.matmul(elim_mtx, U)
        L = np.matmul(L, elim_mtx_inv)
    
    return L, U

In [4]:
A = 10*np.random.rand(4, 4) - 5
A

array([[-2.52181977,  0.07872749,  4.86001156,  2.07827473],
       [-2.09539131,  3.08914232, -2.87146667, -3.00133445],
       [ 3.24887478,  4.60332524, -1.99862347, -4.1464174 ],
       [-1.38667262,  4.01625566,  0.65444065, -3.15511555]])

In [5]:
L, U = LU(A)

print(f"Lower:\n{L}\n\nUpper:\n{U}")

Lower:
[[ 1.          0.          0.          0.        ]
 [ 0.83090447  1.          0.          0.        ]
 [-1.2883057   1.55594399  1.          0.        ]
 [ 0.54986983  1.31392993  0.47029909  1.        ]]

Upper:
[[-2.52181977  0.07872749  4.86001156  2.07827473]
 [ 0.          3.0237273  -6.90967199 -4.72818221]
 [ 0.          0.         15.01361973  5.88782247]
 [ 0.          0.          0.         -0.85443358]]


In [6]:
np.allclose(np.matmul(L, U), A)

True

### Inverting a matrix, for real

In [7]:
def invert_lower_triangular_matrix(L: np.ndarray):
    """
    Computes the inverse of a lower triangular matrix.

    Args:
        L (np.ndarray): A square lower triangular matrix of shape (n, n). 
                        It must have non-zero diagonal elements for the 
                        inversion to succeed.

    Returns:
        np.ndarray: The inverse of the lower triangular matrix L, with
                        shape (n, n).
    """
    n = L.shape[0]
    G = np.eye(n)
    D = np.copy(L)
    
    for step in range(1, n):
        elim_mtx, _ = elimination_matrix(D, step=step)
        G = np.matmul(elim_mtx, G)
        D = np.matmul(elim_mtx, D)
        
    D_inv = np.eye(n)/np.diagonal(D)   # NumPy performs this operation elementwise
    
    return np.matmul(D_inv, G)

In [8]:
def invert(A: np.ndarray):
    """
    Computes the inverse of a square matrix using its LU decomposition.

    Args:
        A (np.ndarray): A square matrix of shape (n, n). The matrix must be 
                        non-singular (invertible) for the inversion to succeed.

    Returns:
        np.ndarray: The inverse of the input matrix A, with shape (n, n).
    """
    L, U = LU(A)
    L_inv = invert_lower_triangular_matrix(L)
    U_inv = invert_lower_triangular_matrix(U.T).T
    return np.matmul(U_inv, L_inv)

In [9]:
A = np.random.rand(3, 3)
A_inv = invert(A)

print(f"A:\n{A}\n\nA⁻¹:\n{A_inv}\n\nAA⁻¹:\n{np.matmul(A, A_inv)}")

A:
[[0.11622029 0.85468856 0.01935418]
 [0.25573164 0.4147443  0.2831796 ]
 [0.72534232 0.17804492 0.49761005]]

A⁻¹:
[[ 1.95009732 -5.27473606  2.92589579]
 [ 0.97712919  0.54758462 -0.34962383]
 [-3.19218031  7.49280414 -2.1302369 ]]

AA⁻¹:
[[ 1.00000000e+00 -2.89495728e-16  6.08127525e-17]
 [ 2.25713230e-16  1.00000000e+00  6.77391599e-18]
 [ 3.99707490e-16 -1.57175183e-15  1.00000000e+00]]


In [10]:
for _ in range(1000):
    n = np.random.randint(1, 10)
    A = np.random.rand(n, n)
    A_inv = invert(A)
    if not np.allclose(np.matmul(A, A_inv), np.eye(n), atol=1e-5):
        print("Test failed.")

### How to actually invert matrices

In [11]:
A = np.random.rand(3, 3)
A_inv = np.linalg.inv(A)

print(f"A:\n{A}\n\nNumPy's A⁻¹:\n{A_inv}\n\nAA⁻¹:\n{np.matmul(A, A_inv)}")

A:
[[0.51874704 0.77117844 0.81353309]
 [0.05266504 0.2246453  0.8597507 ]
 [0.39976704 0.62327768 0.67388369]]

NumPy's A⁻¹:
[[ 47.3604734    1.55547434 -59.15951565]
 [-37.96561146  -2.99963689  49.66023344]
 [  7.01895919   1.85162422  -9.35189695]]

AA⁻¹:
[[ 1.00000000e+00  9.67909512e-17  3.50763709e-15]
 [ 1.68319122e-16  1.00000000e+00  1.13166915e-15]
 [-2.86189018e-15  2.74531349e-16  1.00000000e+00]]


In [12]:
from timeit import timeit


n_runs = 100
size = 100
A = np.random.rand(size, size)

t_inv = timeit(lambda: invert(A), number=n_runs)
t_np_inv = timeit(lambda: np.linalg.inv(A), number=n_runs)


print(f"Our invert:              \t{t_inv} s")
print(f"NumPy's invert:          \t{t_np_inv} s")
print(f"Performance improvement: \t{t_inv/t_np_inv} times faster")

Our invert:              	3.4115849529916886 s
NumPy's invert:          	0.016944584000157192 s
Performance improvement: 	201.33778161565016 times faster


## Determinants in practice

### The recursive way

In [13]:
def det(A: np.ndarray):
    """
    Recursively computes the determinant of a square matrix A.

    Args:
        A (np.ndarray): A square matrix of shape (n, n) for which the 
        determinant is to be calculated.

    Returns:
        float: The determinant of matrix A.

    Raises:
        ValueError: If A is not a square matrix.
    """

    n, m = A.shape
    
    # making sure that A is a square matrix
    if n != m:
        raise ValueError("A must be a square matrix.")
        
    if n == 1:
        return A[0, 0]
    
    else:
        return sum([(-1)**j*A[0, j]*det(np.delete(A[1:], j, axis=1)) for j in range(n)])

In [14]:
A = np.array([[1, 2],
              [3, 4]])

In [15]:
det(A)    # should be -2

np.int64(-2)

In [16]:
from timeit import timeit

A = np.random.rand(10, 10)
t_det = timeit(lambda: det(A), number=1)

print(f"The time it takes to compute the determinant of a 10 x 10 matrix: {t_det} seconds")

The time it takes to compute the determinant of a 10 x 10 matrix: 17.707615608000197 seconds


### How to actually compute determinants

In [17]:
def fast_det(A: np.ndarray):
    """
    Computes the determinant of a square matrix using LU decomposition.
    
    Args:
        A (np.ndarray): A square matrix of shape (n, n) for which the determinant 
                         needs to be computed. The matrix must be non-singular (invertible).

    Returns:
        float: The determinant of the matrix A..
    """

    L, U = LU(A)
    return np.prod(np.diag(U))

In [18]:
A = np.random.rand(10, 10)


t_fast_det = timeit(lambda : fast_det(A), number=1)
print(f"The time it takes to compute the determinant of a 10 x 10 matrix: {t_fast_det} seconds")

The time it takes to compute the determinant of a 10 x 10 matrix: 0.0003277480136603117 seconds


In [19]:
print(f"Recursive determinant:   \t{t_det} s")
print(f"LU determinant:          \t{t_fast_det} s")
print(f"Performance improvement: \t{t_det/t_fast_det} times faster")

Recursive determinant:   	17.707615608000197 s
LU determinant:          	0.0003277480136603117 s
Performance improvement: 	54028.140125825215 times faster


## Problems

**Problem 3.** Before we wrap this chapter up, let's go back to the definition of determinants. Even though we have lots of reasons against using the determinant formula, we have one for it: it is a good exercise, and implementing it will deepen your understanding. So, in this problem, you are going to build

$$
    \det A = \sum_{\sigma \in S_n} \mathrm{sign}(\sigma) a_{\sigma(1)1} \dots a_{\sigma(n)n},
$$

one step at a time.

*(i)* Implement a function that, given an integer $ n $, returns all permutations of the set $ \{0, 1, \dots, n-1\} $. Represent each permutation $ \sigma $ as a list. For example, 

```
[2, 0, 1]
```

would represent the permutation $ \sigma $, where $ \sigma(0) = 2, \sigma(1) = 0 $, and $ \sigma(2) = 1 $.

*(ii)* Let $ \sigma \in S_n $ be a permutation of the set $ {0, 1, \dots, n-1} $. Its *inversion number* is defined by

$$
    \mathrm{inversion}(\sigma) = \big| \{ (i, j): i < j \text{ and } \sigma(i) > \sigma(j) \} \big|,
$$

where $ |\cdot| $ denotes the number of elements in the set. Essentially, inversion describes the number of times a permutation reverses the order of a pair of numbers.

Turns out, the sign of $ \sigma $ can be written as

$$
    \mathrm{sign}(\sigma) = (-1)^{\mathrm{inversion}(\sigma)}.
$$

Implement a function that first calculates the inversion number, then the sign of an arbitrary permutation. (Permutations are represented like in the previous problem.)

*(iii)* Put the solutions for Problem 1. and Problem 2. together and calculate the determinant of a matrix using the permutation formula. What do you think the time complexity of this algorithm is?

### Solutions

**Problem 3.** *(i)*

In [20]:
from copy import deepcopy


def permutations(n: int):
    if n == 0:
        return [[0]]
    else:
        prev_permutations = permutations(n - 1)
        
        new_permutations = []
        
        for p in prev_permutations:
            for i in range(len(p)+1):
                p_new = deepcopy(p)
                p_new.insert(i, n)
                new_permutations.append(p_new)
                
        return new_permutations

*(ii)*

In [21]:
from itertools import product


def inversion(permutation: list):
    n = len(permutation)
    inversions = sum([1 for i, j in product(range(n), range(n)) if i < j and permutation[i] > permutation[j]])
    return inversions

In [22]:
def sign(permutation: list):
    i = inversion(permutation)
    return (-1)**i

*(iii)*

In [23]:
def permutation_formula(A: np.ndarray):
    n, _ = A.shape
    S_n = permutations(n-1)
    determinant = sum([sign(p)*np.prod([A[p[i], i] for i in range(n)]) for p in S_n])
    return determinant